In [1]:
def emails_between(suspects):
    
    

SyntaxError: expected '(' (1515693898.py, line 1)

In [22]:
import os
from email.parser import Parser
from typing import List, Dict

def get_emails_between(suspects):
    emails_between = []
    email_directory = "enron_email_corpus"
    for root, dirs, files in os.walk(email_directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                try:
                    msg = Parser().parse(file)
                except Exception as e:
                    print(f"Failed to parse {file_path}: {e}")
                    continue

                sender = msg.get('From')
                recipients = msg.get('To')

                if sender and recipients:
                    sender_account = None
                    recipient_accounts = []

                    for account, addresses in suspects.items():
                        for address in addresses:
                            if address.lower() in sender.lower():
                                sender_account = account
                            if address.lower() in recipients.lower():
                                recipient_accounts.append(account)

                    if sender_account and recipient_accounts and len(recipient_accounts) <= 20:
                        emails_between.append(msg)
    return emails_between

# # Example usage:
# red_flags = {
#     "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
#     "skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
# }

# email_directory = "enron_email_corpus"
# emails = get_emails_between(red_flags)
# # print(emails)
# print(f"Found {len(emails)} emails between suspects.")

# Example usage

red_flags = {
    "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
    "skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
}

emails = get_emails_between(red_flags)
# for email in emails_between[:10]:  # Print details of the first 10 emails to check
#     print(f"From: {email['From']}, To: {email['To']}, Message ID: {email['Message'].get('Message-ID')}")

print(f"Found {len(emails)} emails between suspects.")



Found 4 emails between suspects.


In [5]:
import os
from email.parser import Parser

def emails_between(suspects):
    """
    Parses the Enron email corpus and returns a list of email objects that were sent between the specified suspects.

    Parameters:
    suspects (dict): A dictionary where keys are account names, and values are lists of email addresses associated with that account.

    Returns:
    list: A list of email.message.Message objects representing emails sent between the suspects.
    """
    # Initialize an empty list to store the email objects
    email_objects = []
    
    # Define the root directory of the Enron email corpus
    root_dir = "enron_email_corpus"
    
    # Parser for reading email files
    parser = Parser()
    
    # Iterate over the suspects' directories
    for suspect in suspects:
        suspect_dir = os.path.join(root_dir, suspect)
        
        # Walk through the suspect's directory
        for root, dirs, files in os.walk(suspect_dir):
            for file in files:
                try:
                    # Construct the file path
                    file_path = os.path.join(root, file)
                    with open(file_path, "r", encoding="utf-8") as f:
                        email_data = f.read()
                    
                    # Parse the email content
                    email_msg = parser.parsestr(email_data)
                    
                    # Extract 'From', 'To' and 'Subject' fields
                    from_addr = email_msg['From']
                    to_addrs = email_msg['To']
                    if to_addrs:
                        to_addrs_list = to_addrs.split(',')
                        
                        # Check if the email is between suspects
                        if from_addr in sum(suspects.values(), []) and any(to_addr in sum(suspects.values(), []) for to_addr in to_addrs_list) and len(to_addrs_list) <= 20:
                            email_objects.append(email_msg)
                
                except Exception as e:
                    # Skip the email if any error occurs
                    continue
    
    return email_objects

# Example usage
red_flags = {
    "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
    "skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
}

# Call the function with the 'red_flags' dictionary
emails = emails_between(red_flags)
# Now 'emails' will contain the email objects of interest.


In [27]:
import os
from email.parser import Parser

def emails_between(suspects):
    parser = Parser()
    emails = []

    def is_suspect(address):
        for emails in suspects.values():
            if address in emails:
                return True
        return False

    def process_email(file_path):
        try:
            with open(file_path) as f:
                email = parser.parse(f)
            from_address = email['From']
            to_addresses = email['To']
            if to_addresses:
                to_addresses = to_addresses.split(',')
                if len(to_addresses) <= 20 and is_suspect(from_address) and any(is_suspect(addr.strip()) for addr in to_addresses):
                    emails.append(email)
        except Exception as e:
            print(f"Skipped an email due to parsing error: {e}")

    for user_dir in suspects.keys():
        user_path = os.path.join("enron_email_corpus", user_dir)
        for root, dirs, files in os.walk(user_path):
            for file in files:
                process_email(os.path.join(root, file))

    return emails

# Example usage
red_flags = {
    "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
    "skilling-j": ["jeff.skilling@enron.com", "skilling@enron.com"]
}

# Call the function with the suspects dictionary
# emails_found = emails_between

# Call the function with the 'red_flags' dictionary
emails = emails_between(red_flags)
# Now 'emails' will contain the email objects of interest.



In [33]:
import os
from email.parser import Parser

def emails_between(suspects):
    email_objects = []
    root_dir = "/Users/devanshbajaj/UCC Modules/PythonLab/enron_email_corpus/maildir"
    parser = Parser()
    
    for suspect in suspects:
        suspect_dir = os.path.join(root_dir, suspect)
        print(f"Checking directory: {suspect_dir}")  # Debugging print
        
        if not os.path.exists(suspect_dir):
            print(f"Directory not found: {suspect_dir}")
            continue
        
        for root, dirs, files in os.walk(suspect_dir):
            for file in files:
                file_path = os.path.join(root, file)
                
                try:
                    with open(file_path, "r", encoding="utf-8") as f:
                        email_data = f.read()

                    email_msg = parser.parsestr(email_data)
                    from_addr = email_msg['From']
                    to_addrs = email_msg.get('To', '')
                    to_addrs_list = to_addrs.split(',')
                    
                    if from_addr in sum(suspects.values(), []) and any(to_addr.strip() in sum(suspects.values(), []) for to_addr in to_addrs_list) and len(to_addrs_list) <= 20:
                        email_objects.append(email_msg)
                
                except Exception as e:
                    print(f"Error parsing {file_path}: {e}")  # Print out the error

    print(f"Total emails found: {len(email_objects)}")  # Debugging print
    return email_objects

# Adjust the suspects dictionary and path as needed before running the function.
# Example usage
red_flags = {
    "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
    "skilling-j": ["jeff.skilling@enron.com", "skilling@enron.com"]
}

# Call the function with the suspects dictionary
# emails_found = emails_between

# Call the function with the 'red_flags' dictionary
emails = emails_between(red_flags)
# Now 'emails' will contain the email objects of interest.




Checking directory: /Users/devanshbajaj/UCC Modules/PythonLab/enron_email_corpus/maildir/lay-k
Error parsing /Users/devanshbajaj/UCC Modules/PythonLab/enron_email_corpus/maildir/lay-k/.DS_Store: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte
Error parsing /Users/devanshbajaj/UCC Modules/PythonLab/enron_email_corpus/maildir/lay-k/inbox/533: 'utf-8' codec can't decode byte 0xbf in position 311: invalid start byte
Error parsing /Users/devanshbajaj/UCC Modules/PythonLab/enron_email_corpus/maildir/lay-k/inbox/333: 'utf-8' codec can't decode byte 0xbf in position 501: invalid start byte
Error parsing /Users/devanshbajaj/UCC Modules/PythonLab/enron_email_corpus/maildir/lay-k/inbox/15: 'utf-8' codec can't decode byte 0xe7 in position 334: invalid continuation byte
Error parsing /Users/devanshbajaj/UCC Modules/PythonLab/enron_email_corpus/maildir/lay-k/sent/.DS_Store: 'utf-8' codec can't decode byte 0x87 in position 27: invalid start byte
Checking directory: /Users/de

In [45]:
import os
from email.parser import Parser
# from typing import List, Dict

def emails_between(suspects):
    emails_between = []
    email_directory = "enron_email_corpus"
    
    parser = Parser()

    for root, dirs, files in os.walk(email_directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                    msg = parser.parse(file)

                sender = msg.get('From')
                recipients = msg.get('To')

                if sender and recipients:
                    recipient_emails = set([email.strip() for email in recipients.split(',') if email.strip() != ''])
                    
                    # Check if the sender is a suspect
                    sender_match = any(sender.lower() == address.lower() for account, addresses in suspects.items() for address in addresses)
                    
                    # Check if any recipients are suspects and limit to 20 or fewer unique recipients
                    recipient_matches = any(recipient.lower() in sum(suspects.values(), []) for recipient in recipient_emails)
                    
                    if sender_match and recipient_matches and len(recipient_emails) <= 20:
                        email_detail = {'From': sender, 'To': list(recipient_emails), 'Message': msg}
                        emails_between.append(email_detail)
            except Exception as e:
                print(f"Failed to parse {file_path}: {e}")
                continue

    return emails_between

# Example usage:
red_flags = {
    "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
    "skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
}

emails = emails_between(red_flags)
for email in emails[:10]:  # Print details of the first 10 emails to check
    print(f"From: {email['From']}, To: {email['To']}, Message ID: {email['Message'].get('Message-ID')}")

print(f"Found {len(emails)} emails between suspects.")


From: skilling@enron.com, To: ['jeff.skilling@enron.com'], Message ID: <6926943.1075840150359.JavaMail.evans@thyme>
Found 1 emails between suspects.


In [ ]:
import os
from email.parser import Parser

def emails_between(suspects):
    emails_between = []
    email_directory = "enron_email_corpus"
    
    parser = Parser()
    
    
    
    

In [44]:
import os
from email.parser import Parser
from typing import List, Dict

def get_emails_between(suspects: Dict[str, List[str]]):
    emails_between = []
    email_directory = "enron_email_corpus"
    
    parser = Parser()

    for root, dirs, files in os.walk(email_directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                    msg = parser.parse(file)

                sender = msg.get('From')
                recipients = msg.get('To')

                if sender and recipients:
                    recipient_emails = set(email.strip().lower() for email in recipients.split(',') if email.strip() != '')
                    
                    # Initialize variables to store the sender and recipients' suspect names
                    sender_suspect = None
                    recipient_suspects = set()
                    
                    # Determine if the sender is a suspect
                    for suspect, addresses in suspects.items():
                        if any(sender.lower() == address.lower() for address in addresses):
                            sender_suspect = suspect
                            break
                    
                    # Determine which recipients are suspects
                    for suspect, addresses in suspects.items():
                        if any(recipient in [a.lower() for a in addresses] for recipient in recipient_emails):
                            recipient_suspects.add(suspect)
                    
                    # Ensure the email exchange is between different suspects
                    if sender_suspect and recipient_suspects and sender_suspect in recipient_suspects:
                        recipient_suspects.remove(sender_suspect)  # Remove the sender from the recipient suspects set
                    
                    if sender_suspect and recipient_suspects and len(recipient_emails) <= 20:
                        email_detail = {'From': sender, 'To': list(recipient_emails), 'Message': msg}
                        emails_between.append(email_detail)
            except Exception as e:
                print(f"Failed to parse {file_path}: {e}")
                continue

    return emails_between

# Example usage

red_flags = {
    "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
    "skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
}

emails = emails_between(red_flags)
for email in emails[:10]:  # Print details of the first 10 emails to check
    print(f"From: {email['From']}, To: {email['To']}, Message ID: {email['Message'].get('Message-ID')}")

print(f"Found {len(emails)} emails between suspects.")



AttributeError: 'NoneType' object has no attribute 'get'

In [41]:
import os
from email.parser import Parser
from typing import Dict, List

def emails_between(suspects: Dict[str, List[str]]):
    email_objects = []
    root_dir = "enron_email_corpus/maildir"  # Adjust the path as necessary
    
    # Create a reverse lookup dictionary for efficient email address to suspect mapping
    email_to_suspect = {}
    for suspect, emails in suspects.items():
        for email in emails:
            email_to_suspect[email.lower()] = suspect
    
    # Initialize the email parser
    parser = Parser()
    
    # Walk through the directory structure
    for root, dirs, files in os.walk(root_dir):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    email_msg = parser.parse(file)
                
                # Extract the sender and recipients
                sender = email_msg['From']
                recipients = email_msg['To']
                if not sender or not recipients:
                    continue
                
                # Check if the sender is a suspect
                sender_suspect = email_to_suspect.get(sender.lower())
                if not sender_suspect:
                    continue
                
                # Split and clean the recipient list, then check if any are suspects
                recipient_list = [r.strip() for r in recipients.split(',') if r.strip()]
                if len(recipient_list) > 20:
                    continue  # Skip emails with more than 20 recipients
                
                recipient_suspects = set(email_to_suspect.get(r.lower()) for r in recipient_list)
                recipient_suspects.discard(None)  # Remove non-suspects
                
                # Check for emails where both sender and at least one recipient are suspects and not the same person
                if sender_suspect in recipient_suspects:
                    email_objects.append(email_msg)
            
            except Exception as e:
                # Skip any files that cause parsing errors
                continue
    
    return email_objects

# Example usage
red_flags = {
    "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
    "skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
}

# This will process the specified email directory and find emails between the suspects
emails = emails_between(red_flags)
print(f"Found {len(emails)} emails between the suspects.")

for email in emails[:10]:  # Print details of the first 10 emails to check
    print(f"From: {email['From']}, To: {email['To']}, Message ID: {email['Message'].get('Message-ID')}")
print(f"Found {len(emails)} emails between the suspects.")


Found 1 emails between the suspects.


AttributeError: 'NoneType' object has no attribute 'get'